In [ ]:
!pip install robin_stocks pandas

In [1]:
import robin_stocks.robinhood as r
import pandas as pd

email = input("Please enter your Robinhood email address: ")
login = r.login(email)

# Get all option orders
all_orders = r.orders.get_all_option_orders()

df = pd.DataFrame(all_orders)

df.to_csv('all_options_orders.csv')


Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...


get stock orders

This is getting all option orders (since the open orders doesn't return all the data it seems)

In [10]:
import robin_stocks.robinhood as r
import pandas as pd

# email = input("Please enter your Robinhood email address: ")
login = r.login(email)

# # # Get all option orders
# all_orders = r.orders.get_all_option_orders()


# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_orders)

df = df[df['state'] == 'filled']

legs_data = df['legs'].apply(pd.Series)

#print(legs_data)

# Extract the strike_price
#strike_prices = [item['strike_price'] for item in legs_data if isinstance(item, dict)]

# Extract the strike_price
df['strike_price'] = df['legs'].apply(lambda x: [float(leg['strike_price']) for leg in x] if x else [])
df['expiration_date'] = df['legs'].apply(lambda x: x[0]['expiration_date'] if x and isinstance(x, list) and len(x) > 0 else None)

# Convert 'created_at' to datetime format with full precision
df['created_at'] = pd.to_datetime(df['created_at']).dt.strftime('%Y-%m-%d %H:%M:%S')
# df = df[pd.to_datetime(df['created_at']).dt.year == 2024]

# Convert 'average_net_premium_paid' to float
df['premium'] = df['premium'].astype(float)
# # Update the processed_premium value based on the direction

# Calculate the total of 'average_net_premium_paid'
df['cost'] = df['price'].astype(float)  * df['quantity'].astype(float) * 100


df['cost'] = df.apply(
    lambda x: -x['cost'] if x['direction'] == 'debit' else x['cost'],
    axis=1
)

total = df['cost'].sum()

# Add a new row with the total
#df.loc['Total'] = pd.Series({'average_net_premium_paid': total})
df = df.round(2)
print(total)
print(df['premium'].count())
print(df['quantity'].astype(float).sum() )


# Specify the new column order
new_order = ['created_at', 'chain_symbol', 'expiration_date', 'strike_price', 'direction', 'price', 
            'quantity', 'premium', 'cost','opening_strategy', 'closing_strategy' ]
# Reorder the columns
df = df.reindex(columns=new_order)

# Count rows where 'opening_strategy' is not None
total_opening_strategy_count = df['opening_strategy'].notna().sum()
total_closing_strategy_count = df['closing_strategy'].notna().sum()

#Print the counts
print(f"Opening strategy rows: {total_opening_strategy_count}")
print(f"Closing strategy rows: {total_closing_strategy_count}")
# Sort the DataFrame
df = df.sort_values(by=['created_at','chain_symbol', 'expiration_date'])
# Save the DataFrame to a CSV file
df.to_csv('../output/rh_rorders_export.csv', index=False)

-3842.0
877
1626.0
Opening strategy rows: 544
Closing strategy rows: 336


comparing the two methods of getting option orders (rh_export vs rorders)  

In [3]:

df1 = pd.read_csv('../output/options_orders_parsed.csv')
df2 = pd.read_csv('../output/rh_rorders_export.csv')

# Initialize a counter for differences
difference_count = 0
df2 = df2.drop(index=148)  
# Loop through the DataFrames and compare the 'cost' column row by row
for index in range(min(len(df1), len(df2))):

    if pd.notna(df1['cost'].iloc[index]) and pd.notna(df2['cost'].iloc[index]):  
        print(f"index {index}")
        print(int(round(df1['cost'].iloc[index],0)))
        print(int(round(df2['cost'].iloc[index],0)))
        print(int(round(df1['cost'].iloc[index],0)) == int(round(df2['cost'].iloc[index],0)))
    else:
        print("One of the values is NaN")  # Handle NaN cases
  



index 0
200
200
True
index 1
-780
-780
True
index 2
586
586
True
index 3
-92
-92
True
index 4
-103
-103
True
index 5
157
157
True
index 6
-54
-54
True
index 7
-60
-60
True
index 8
-14
-14
True
index 9
-30
-30
True
index 10
-40
-40
True
index 11
-12
-12
True
index 12
-27
-27
True
index 13
-52
-52
True
index 14
-16
-16
True
index 15
15
15
True
index 16
-18
-18
True
index 17
54
54
True
index 18
-30
-30
True
index 19
7
7
True
index 20
-69
-69
True
index 21
-7
-7
True
index 22
-7
-7
True
index 23
-4
-4
True
index 24
-31
-31
True
index 25
-35
-35
True
index 26
-39
-39
True
index 27
170
170
True
index 28
-25
-25
True
index 29
-96
-96
True
index 30
-11
-11
True
index 31
5
5
True
index 32
384
384
True
index 33
-480
-480
True
index 34
402
402
True
index 35
-126
-126
True
index 36
-83
-83
True
index 37
6
6
True
index 38
-59
-59
True
index 39
21
21
True
index 40
-10
-10
True
index 41
-30
-30
True
index 42
-20
-20
True
index 43
-25
-25
True
index 44
-115
-115
True
index 45
120
120
True
index 46
-10